In [6]:
import csv
import plotly.graph_objects as go
import plotly.io as pio
import emblaze
from gensim.models import Word2Vec
import numpy as np
import csv
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pacmap
import json
from collections import defaultdict
import re
import emblaze
from emblaze.utils import Field, ProjectionTechnique

In [10]:
model_5000 = Word2Vec.load("../Data/Models/word2vec_epoch_5000.model")
#model_200 = Word2Vec.load("../Data/Models/word2vec_epoch_200_min_1.model")
model_1000 = Word2Vec.load("../Data/Models/word2vec_epoch_1000_min_1.model")
#model_3000 = Word2Vec.load("../Data/Models/word2vec_epoch_3000_min_1.model")



In [2]:

with open('../Data/sequences.csv', 'r') as f:
    reader = csv.reader(f)
    sequences = list(reader)
    
address_count_seq = {}
for seq in sequences:
    for place in seq:
        if place in address_count_seq:
            address_count_seq[place] += 1
        else:
            address_count_seq[place] = 1

address_count_seq

def label_function_address_count(labels, station_names): 
    for station_name in station_names:
            # Check if the station_name exists in address_coordinates dictionary
            if station_name in address_count_seq:
                if address_count_seq[station_name] == 1:
                    labels.append(address_count_seq[station_name])
                else: 
                 labels.append("")
            #     labels.append(address_count_seq[station_name])
            # else: 
            #     labels.append("")
    return labels

labels = label_function_address_count([], model_1000.wv.index_to_key)
# Convert labels to numerical format
label_encoder = LabelEncoder()
label_encoder.fit(labels)
numeric_labels = label_encoder.transform(labels)

# X is an n x k array, Y is a length-n array
X, Y = model_1000.wv.vectors, labels

# Represent the high-dimensional embedding
emb = emblaze.Embedding({Field.POSITION: X, Field.COLOR: Y})
# Compute nearest neighbors in the high-D space (for display)
emb.compute_neighbors(metric='cosine')

# Generate UMAP 2D representations - you can pass UMAP parameters to project()
variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.UMAP) for _ in range(10)
])
# Compute neighbors again (to indicate that we want to compare projections)
variants.compute_neighbors(metric='euclidean')

w = emblaze.Viewer(embeddings=variants)
w

Viewer(colorScheme='tableau', data={'data': [{'_format': 'compressed', '_idtype': 'u2', '_length': 40581, 'ids…

In [8]:
import json
f = open('../Data/data.json')
data = json.load(f)

a = open('../Data/added_data.json')
added_data = json.load(a)

In [14]:
import re

#retrieve zipcode
def getZip(info):
    pattern = '\d{4}'
    match = re.search(pattern, info)
    if match is not None:
        return match.group()
    else :
        return

# # Extract zip codes and corresponding coordinates
#zip_coordinates = defaultdict(list)
address_coordinates = {}
wrong_addresses = []
# we have some json objects where the address exists but the info is null. This happens in 1602 cases (assume this is where the geolocator failed)
for address, info in data.items():
    if info is not None: 
        zip_code = getZip(info['address'])  # Extract zip code from address could be empty
        if zip_code is not None:
            if 1000 <= int(zip_code) < 4999: #for now removing all failed zipcodes in Jylland
                 #zip_coordinates[zip_code].append((address,info["latitude"], info["longitude"]))
                 address_coordinates[address] = zip_code
            else: 
                wrong_addresses.append(address) #go through later
        else:
            address_coordinates[address] = 0000 #no zipcode was found
    else:
        address_coordinates[address] = 0000 #no info was found on the address?

#run through the list of wrong addresses and try to find them in the added_data.json file to get the "correct" zipcode
for address in wrong_addresses:
    #find in json file
    if address in added_data:
        zip = getZip(added_data[address]['address'])
        address_coordinates[address] = zip

def label_function_zipcodes(labels, station_names): 
    for station_name in station_names:
    # Check if the station_name exists in address_coordinates dictionary
        if station_name in address_coordinates:
            zip = address_coordinates[station_name]
            if zip is not None:
                labels.append(zip)
            else: 
                labels.append("unknown")
        else:
            labels.append("unknown")
    return labels

labels = label_function_zipcodes([], model_5000.wv.index_to_key)
# Convert labels to numerical format
label_encoder = LabelEncoder()
label_encoder.fit(labels)
numeric_labels = label_encoder.transform(labels)

# X is an n x k array, Y is a length-n array
X, Y = model_5000.wv.vectors, labels

# Represent the high-dimensional embedding
emb = emblaze.Embedding({Field.POSITION: X, Field.COLOR: Y})
# Compute nearest neighbors in the high-D space (for display)
emb.compute_neighbors(metric='cosine')

# Generate UMAP 2D representations - you can pass UMAP parameters to project()
variants = emblaze.EmbeddingSet([
    emb.project(method=ProjectionTechnique.tSNE) for _ in range(10)
])
# Compute neighbors again (to indicate that we want to compare projections)
variants.compute_neighbors(metric='euclidean')

w = emblaze.Viewer(embeddings=variants)
w


AttributeError: type object 'ProjectionTechnique' has no attribute 'tSNE'